<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo II: Vectores Palabra (Word Embeddings)

# Word Embeddings



## Motivación

### El gran problema de Bag of Words

Pensemos en estas 3 frases como documentos:

- $doc_1$: `¡Buenísimo el croissant!`
- $doc_2$: `¡Estuvo espectacular ese pan francés!`
- $doc_3$: `!Buenísima esa pintura!`

Sabemos $doc_1$ y $doc_2$ hablan de lo mismo 🍞🍞👌 y que $doc_3$ 🎨 no tiene mucho que ver con los otros.

Supongamos que queremos ver que tan similares son ambos documentos. 

Para esto, generamos un modelo `Bag of Words` sobre el documento. Es decir, transformamos cada palabra a un vector one-hot y luego los sumamos por documento. 

Además, omitimos algunas stopwords y consideramos pan frances como un solo token.

$$v = \{buenísima, croissant, estuvo, espectacular, pan\ francés, pintura\}$$

Entonces, el $\vec{doc_1}$ quedará:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} =
  \begin{bmatrix}1 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix}$$

El $\vec{doc_2}$ quedará:

$$\begin{bmatrix}0 \\ 0 \\ 1 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1\\ 0\end{bmatrix} = 
  \begin{bmatrix}0 \\ 0 \\ 1 \\ 1 \\ 1\\ 0\end{bmatrix}$$

Y el $\vec{doc_3}$: 

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} + 
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix} =
  \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix}$$



**¿Cuál es el problema?**

`buenísima` $\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `espectacular` $ \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$ representan ideas muy similares. Por otra parte, sabemos que `croissant` $\begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `pan francés` $\begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1 \\0\end{bmatrix}$ se refieren al mismo objeto. Pero en este modelo, estos **son totalmente distintos**. Es decir, los vectores de las palabras que `buenísima` y `espectacular` son tan distintas como `croissant` y `pan francés`. Esto evidentemente, repercute en la calidad de los modelos que creamos a partir de nuestro Bag of Words.

Ahora, si queremos ver que documento es mas similar a otro usando distancia euclidiana, veremos que:

$$d(doc_1, doc_2) = 2.236$$
$$d(doc_1, doc_3) = 1.414$$

Es decir, $doc_1$ se parece mas a $doc_3$ aunque nosotros sabemos que $doc_1$ y $doc_2$ nos están diciendo lo mismo!


Nos gustaría que eso no sucediera. Que existiera algún método que nos permitiera hacer que palabras similares tengan representaciones similares. Y que con estas, representemos mejor a los documentos.


--------------------

### Hipótesis Distribucional

Estamos buscando algún enfoque que nos permita representar las palabras de forma no aislada, si no como algo que además capture el significado de esta.

Pensemos un poco en la **hipótesis distribucional**. Esta plantea que:

    "Palabras que ocurren en contextos iguales tienden a tener significados similares." 

O equivalentemente,

    "Una palabra es caracterizada por la compañía que esta lleva."

Esto nos puede hacer pensar que podríamos usar los contextos de las palabras para generar vectores que describan mejor dichas palabras: en otras palabras, los `Distributional Vectors`.

--------------------


### Word-Context Matrices

Es una matriz donde cada celda $(i,j)$ representa la co-ocurrencia entre una palabra objetivo/centro $w_i$ y un contexto $c_j$. El contexto son las palabras dentro de ventana de tamaño $k$ que rodean la palabra central. 

Cada fila representa a una palabra a través de su contexto. Como se puede ver, ya no es un vector one-hot, si no que ahora contiene mayor información.

El tamaño de la matriz es el tamaño del vocabulario $V$ al cuadrado. Es decir $|V|*|V|$.

<img src="./Figures/distributionalSocher.png" alt="Word-context matrices" style="width: 400px;"/>


**Problema: Creada a partir de un corpus respetable, es gigantezca**. 

Por ejemplo, para $|v| = 100.000$, la matriz tendrá $\frac{100000 * 100000 * 4}{10^9} = 40gb $.

- Es caro mantenerla en memoria 
- Los clasificadores no funcionan tan bien con tantas dimensiones (ver [maldición de la dimensionalidad](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)).

¿Habrá una mejor solución?

---------------------

### Word Embeddings


La idea principal de los Word Embeddings es crear representaciones vectoriales densas y de baja dimensionalidad $(d << |V|)$ de las palabras a partir de su contexto.  Para esto, se usan distintos modelos que emplean redes neuronales *shallow* o poco profundas.

Volvamos a nuestro ejemplo anterior: `buenísima` y `espectacular` ocurren muchas veces en el mismo contexto, por lo que los embeddings que los representan debiesen ser muy similares... :

`buenísima` $\begin{bmatrix}0.32 \\ 0.44 \\ 0.92 \\ .001 \end{bmatrix}$ y `espectacular` $\begin{bmatrix}0.30 \\ 0.50 \\ 0.92 \\ .002 \end{bmatrix}$ versus `croissant`  $\begin{bmatrix}0.77 \\ 0.99 \\ 0.004 \\ .1 \end{bmatrix}$ el cuál es claramente distinto.


Pero, **¿Cómo capturamos el contexto dentro de nuestros vectores?**

- Dependerá del modelo que utilizemos.


##### Word2vec y Skip-gram

Word2Vec es probablemente el paquete de software mas famoso para crear word embeddings. Este nos provee herramientas para crear distintos tipos de modelos, tales como `Skip-Gram` y `Continuous Bag of Word (CBOW)`. En este caso, solo veremos `Skip-Gram`.

**Skip-gram** es una task auxiliar con la que crearemos nuestros embeddings. Esta consiste en que por cada palabra del dataset, predigamos las palabras de su contexto (las palabras presentes en ventana de algún tamaño $k$).

Para resolverla, usaremos una red de una sola capa oculta. Los pesos ya entrenados de esta capa serán los que usaremos como embeddings.

#### Detalles del Modelo

- Como dijimos, el modelo será una red de una sola capa. La capa oculta tendrá una dimensión $d$ la cual nosotros determinaremos. Esta capa no tendrá función de activación. Sin embargo, la de salida si, la cual será una softmax.

- El vector de entrada, de tamaño $|V|$, será un vector one-hot de la palabra que estemos viendo en ese momento.

- La salida, también de tamaño $|V|$, será un vector que contenga la distribución de probabilidad de que cada palabra del vocabulario pertenezca al contexto de la palabra de entrada.

- Al entrenar, se comparará la distribución de los contextos con la suma de los vectores one-hot del contexto real.

<img src="./Figures/skip_gram_net_arch.png" alt="Skip Gram" style="width: 600px;"/>

Nota: Esto es computacionalmente una locura. Por cada palabra de entrada, debemos calcular la probabilidad de aparición de todas las otras. Imaginen el caso de un vocabulario de 100.000 de palabras y de 10000000 oraciones...

La solución a esto es modificar la task a *Negative Sampling*. Esta transforma este problema de $|V|$ clases a uno binario.

### La capa Oculta y los Embeddings

Al terminar el entrenamiento, ¿Qué nos queda en la capa oculta?

Una matriz de $v$ filas por $d$ columnas, la cual contiene lo que buscabamos: Una representación continua de todas las palabras de nuestro vocabualrio.  

**Cada fila de la matriz es un vector que contiene la representación continua una palabra del vocabulario.**


<img src="./Figures/word2vec_weight_matrix_lookup_table.png" alt="Capa Oculta 1" style="width: 400px;"/>

¿Cómo la usamos eficientemente?

Simple: usamos los mismos vectores one-hot de la entrada y las multiplicamos por la matriz:

<img src="./Figures/matrix_mult_w_one_hot.png" alt="Skip Gram" style="width: 400px;"/>






## Entrenar nuestros Embeddings

Para entrenar nuestros embeddings, usaremos el paquete gensim. Este trae una muy buena implementación de `word2vec`.




In [4]:
import re
import pandas as pd
from time import time
from collections import defaultdict
import string
import multiprocessing
import os

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser


In [5]:
# install word2vec
# !pip install gensim

### Cargar el dataset y limpiar



In [6]:
dataset = pd.read_json('Data/data')

In [7]:
dataset_r = dataset.copy(deep=True)

In [8]:
dataset_r.copy()

,author,author_link,title,link,category,subcategory,content,tags,embedded_links,publication_datetime
0,Yerko Roa,/lista/autores/yroa,Colapsa otro segmento de casa que se derrumbó ...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,Noticia en Desarrollo Estamos recopilando m...,[],[],1565778000000
1,Valentina González,/lista/autores/vgonzalez,Policía busca a mujer acusada de matar a su pa...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Detectives de la Policía de Investigaciones ...,"[#parricidio, #PDI, #Pudahuel, #Región Metropo...",[https://media.biobiochile.cl/wp-content/uploa...,1565771820000
2,Felipe Delgado,/lista/autores/fdelgado,Dos detenidos en Liceo de Aplicación: protagon...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Dos detenidos fue el saldo de una serie de i...,"[#Incendio, #Liceo de Aplicación, #Región Metr...",[],1565772480000
3,Matías Vega,/lista/autores/mvega,Apoyo transversal: Senado aprueba en general p...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,La sala del Senado aprobó en general el proy...,"[#Inmigración, #Inmigrantes, #Ley, #Migración,...",[https://media.biobiochile.cl/wp-content/uploa...,1565772720000
4,Valentina González,/lista/autores/vgonzalez,Evacuación espontánea en Instituto Nacional po...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,La mañana de este miércoles se produjo una e...,"[#Carabineros, #FFEE, #Gases Lacrimógenos, #In...",[],1565772960000
...,...,...,...,...,...,...,...,...,...,...
26408,Manuel Stuardo,/lista/autores/mstuardo,Naciones Unidas abre proceso de postulaciones ...,https://www.biobiochile.cl/noticias/nacional/c...,nacional,chile,Las Naciones Unidas abrió un proceso de post...,"[#cambio climático, #COP25, #Naciones Unidas, ...",[],1565764200000
26409,Felipe Delgado,/lista/autores/fdelgado,Fernando Astengo chocó en estado de ebriedad e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,El exfutbolista Fernando Astengo protagonizó...,"[#Accidente, #Fernando Astengo, #Peñalolén, #R...",[https://media.biobiochile.cl/wp-content/uploa...,1565767440000
26410,Felipe Delgado,/lista/autores/fdelgado,Detuvieron a hombre que arrojó combustible a u...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-metropolitana,Personal de Carabineros detuvo a un hombre q...,"[#Indigente, #Parque Forestal, #Región Metropo...",[],1565769300000
26411,Nicolás Parra,/lista/autores/nparra,Revelan identidad de 2 de 6 víctimas fatales e...,https://www.biobiochile.cl/noticias/nacional/r...,nacional,region-de-valparaiso,"El intendente de Valparaíso, Jorge Martínez,...","[#derrumbe en valparaíso, #Región de Valparaís...",[],1565771100000


In [9]:
dataset_r.iloc[1,6]

'  Detectives de la Policía de Investigaciones realizan peritajes para detener a una mujer de 45 años, por su presunta responsabilidad en el ataque con un arma cortante contra su propio padre , lo que causó su muerte en la comuna de Pudahuel.  El hecho ocurrió en calle Presidente Truman, cerca de la intersección con Teniente Cruz, cuando, acorde a la declaración del hijo de la víctima y hermano de la victimaria, ambos sostuvieron un enfrentamiento verbal debido a la intensión de Hernan Silva Pérez de vender su casa .  Negocio que habría causado más que molestia en su hija, Tania Silva, quien tras la discusión habría acudido a la cocina de la vivienda para volver con un cuchillo y apuñalar a su padre.  Las primeras diligencias la realizaron carabineros de la 45º comisaría, quienes tomaron declaración al único testigo del crimen, al interior de la vivienda.  El capitán, Carlos Lagos, explicó que la principal hipótesis apunta a una discusión por el dinero de la venta de este inmueble.    

In [11]:
content = dataset['title'] + dataset['content'] 

In [12]:
content.shape

(26413,)

In [13]:
content[0]

'Colapsa otro segmento de casa que se derrumbó en Valparaíso  Noticia en Desarrollo  Estamos recopilando más antecedentes sobre esta noticia, quédate atento a las actualizaciones.    Parte de la estructura restante de la casa que cayó ayer en Valparaíso colapsó en la mañana de este miércoles.  El hecho se produjo a las 10:15 horas, en la esquina de Aldunate con Huito, donde ayer murieron seis personas.  Según información preliminar, no había rescatistas en el lugar, porque las labores se habían suspendido por el peligro de seguir trabajando. No habría lesionados.  '

In [14]:
from collections import Counter

punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv("Data/spanish.txt").values
stopwords = Counter(stopwords.flatten().tolist())


In [15]:
stopwords

Counter({'actualmente': 1,
         'adelante': 1,
         'además': 1,
         'afirmó': 1,
         'agregó': 1,
         'ahora': 1,
         'ahí': 1,
         'al': 1,
         'algo': 1,
         'alguna': 1,
         'algunas': 1,
         'alguno': 1,
         'algunos': 1,
         'algún': 1,
         'alrededor': 1,
         'ambos': 1,
         'ampleamos': 1,
         'ante': 1,
         'anterior': 1,
         'antes': 1,
         'apenas': 1,
         'aproximadamente': 1,
         'aquel': 1,
         'aquellas': 1,
         'aquellos': 1,
         'aqui': 1,
         'aquí': 1,
         'arriba': 1,
         'aseguró': 1,
         'así': 1,
         'atras': 1,
         'aunque': 1,
         'ayer': 1,
         'añadió': 1,
         'aún': 1,
         'bajo': 1,
         'bastante': 1,
         'bien': 1,
         'buen': 1,
         'buena': 1,
         'buenas': 1,
         'bueno': 1,
         'buenos': 1,
         'cada': 1,
         'casi': 1,
         'cerca': 

In [19]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')



In [20]:
def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans('','',punctuation)).lower().split()
    else:
        tokenized_doc = doc.translate(str.maketrans('','',punctuation)).split()
    
    tokenized_doc = [stemmer.stem(token) for token in tokenized_doc if token.lower() not in stopwords]
    return tokenized_doc

In [21]:
cleaned_content = [simple_tokenizer(doc) for doc in content.values]

In [22]:
cleaned_content[14]

['ministr',
 'cubill',
 'extension',
 'par',
 'docent',
 'problem',
 'propuest',
 'ministr',
 'educ',
 'marcel',
 'cubill',
 'refir',
 'a',
 'votacion',
 'realiz',
 'interior',
 'colegi',
 'profesor',
 'decid',
 'manten',
 'par',
 'pes',
 'llam',
 'president',
 'gremi',
 'mari',
 'aguil',
 'a',
 'repleg',
 'entrev',
 'program',
 'expres',
 'bio',
 'bio',
 'radi',
 'ministr',
 'valor',
 'dich',
 'aguil',
 'previ',
 'a',
 'votacion',
 'asegur',
 'efect',
 'posit',
 'vuelt',
 'clas',
 'par',
 'profesor',
 'extiend',
 'seman',
 '255',
 'vot',
 'marc',
 'diferent',
 'sufragi',
 '95',
 'colegi',
 'funcion',
 'normal',
 'baj',
 '490',
 'colegi',
 'qued',
 'par',
 'a',
 'ministeri',
 'segu',
 'negoci',
 'ofrec',
 'consider',
 'mayor',
 'vot',
 'extend',
 'moviliz',
 'ministr',
 'limit',
 'a',
 'esper',
 'resuelv',
 'dialog',
 'intern',
 'llev',
 'a',
 'cab',
 'interior',
 'gremi',
 'line',
 'sostuv',
 'decision',
 'president',
 'colegi',
 'punt',
 'negoci',
 'llam',
 'a',
 'acept',
 'propuest'

### Extracción de Frases

Para crear buenas representaciones, es necesario tambien encontrar conjuntos de palabras que por si solas no tengan mayor significado (como `nueva` y `york`), pero que juntas que representen ideas concretas (`nueva york`). 

Para esto, usaremos el primer conjunto de herramientas de `gensim`: `Phrases` y `Phraser`.

In [23]:
#La condición para que sean considerados es que aparezcan por lo menos 100 veces repetidas.

phrases = Phrases(cleaned_content, min_count = 100, progress_per = 5000)


Ahora, usamos `Phraser` para re-tokenizamos el corpus con los bigramas encontrados. Es decir, juntamos los tokens separados que detectamos como frases.

In [25]:
phrases.vocab

{'colaps': 584,
 'segment': 175,
 'colaps_segment': 1,
 'cas': 17621,
 'segment_cas': 1,
 'derrumb': 536,
 'cas_derrumb': 6,
 'valparais': 2017,
 'derrumb_valparais': 11,
 'notici': 1675,
 'valparais_notici': 1,
 'desarroll': 4340,
 'notici_desarroll': 8,
 'recopil': 201,
 'desarroll_recopil': 1,
 'antecedent': 2320,
 'recopil_antecedent': 25,
 'antecedent_notici': 1,
 'quedat': 12,
 'notici_quedat': 1,
 'atent': 1598,
 'quedat_atent': 1,
 'a': 241340,
 'atent_a': 215,
 'actualiz': 376,
 'a_actualiz': 19,
 'estructur': 624,
 'actualiz_estructur': 1,
 'restant': 200,
 'estructur_restant': 1,
 'restant_cas': 2,
 'cay': 724,
 'cas_cay': 2,
 'cay_valparais': 1,
 'valparais_colaps': 1,
 'mañan': 2853,
 'colaps_mañan': 2,
 'miercol': 5322,
 'mañan_miercol': 159,
 'produj': 1014,
 'miercol_produj': 3,
 'produj_a': 70,
 '1015': 13,
 'a_1015': 4,
 'hor': 8655,
 '1015_hor': 3,
 'esquin': 233,
 'hor_esquin': 2,
 'aldunat': 19,
 'esquin_aldunat': 1,
 'huit': 10,
 'aldunat_huit': 8,
 'mur': 3672,
 

In [27]:
bigram = Phraser(phrases)
sentences = bigram[cleaned_content]

In [28]:
sentences[110]

['ministr',
 'viviend',
 'enfrent',
 'polem',
 'casit',
 'call',
 'oficin',
 'ministr',
 'viviend_urban',
 'cristi',
 'monckeberg',
 'refir',
 'a',
 'polem',
 'reflexion',
 'integr',
 'social',
 'deficit',
 'habitacional',
 'pais',
 'comision',
 'viviend_urban',
 'sen',
 'mayor',
 'propietari',
 'patrimoni',
 'casit',
 'departament',
 'radic',
 'patrimoni',
 'chilen',
 'episodi',
 'entrev',
 'program',
 'peor',
 'juli',
 'ces',
 'rodriguez',
 'remarc',
 'dich',
 'intervencion',
 'larg',
 '14',
 'agost',
 'siqu',
 'sac',
 'context',
 'escuch',
 'grabacion',
 'efect',
 'suen',
 'horror',
 'explic',
 'larg',
 'tem',
 'trat',
 'sen',
 'a',
 'senador',
 'llam_atencion',
 'ministr',
 'viviend',
 'mayor',
 'propietari',
 'casit',
 'departament',
 'pas',
 'instanci',
 'analiz',
 'chilen',
 'cultural',
 'propietari',
 'aspir',
 'a',
 'propietari',
 'algui',
 'sint',
 'ofend',
 'pid',
 'disculp',
 'cas',
 '¿cuant',
 'person',
 'viviend',
 'chil',
 'refir',
 'a',
 'cifr',
 'uso',
 'defend',
 'adm

### Definir el modelo



Primero, como es usual, creamos el modelo. En este caso, usaremos uno de los primero modelos de embeddings neuronales: `word2vec`

Algunos parámetros importantes:

- `min_count`: Ignora todas las palabras que tengan frecuencia menor a la indicada.
- `window` : Tamaño de la ventana. Usaremos 4.
- `size` : El tamaño de los embeddings que crearemos. Por lo general, el rendimiento sube cuando se usan mas dimensiones, pero después de 300 ya no se nota cambio. Ahora, usaremos solo 200.
- `workers`: Cantidad de CPU que serán utilizadas en el entrenamiento.

In [26]:
w2vec = Word2Vec(min_count = 10,
                 window = 4,
                 sample= 6e-5,
                 alpha= 0.03,
                 min_alpha= 0.0007,
                 negative= 0,
                 workers = multiprocessing.cpu_count())

### Construir el vocabulario

Para esto, se creará un conjunto que contendrá (una sola vez) todas aquellas palabras que aparecen mas de `min_count` veces. 

In [29]:
w2vec.build_vocab(sentences, progress_per = 10000)

### Entrenar el Modelo

A continuación, entenaremos el modelo. 
Los parámetros que usaremos serán: 

- `total_examples`: Número de documentos.
- `epochs`: Número de veces que se iterará sobre el corpus.

Es recomendable que tengan instalado `cpython` antes de continuar. Aumenta bastante la velocidad de entrenamiento.


In [30]:
t = time()
w2vec.train(sentences, total_examples = w2vec.corpus_count, epochs=15, report_delay=10 )
print('time: {} mins'.format(round((time() - t)/60,2)))

time: 1.88 mins


In [31]:
w2vec.init_sims(replace=True)

<ipython-input-31-6c363b0058da>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2vec.init_sims(replace=True)


###  Guardar y cargar el modelo

Para ahorrar tiempo, usaremos un modelo preentrenado.

In [33]:
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')

w2vec.save('./pretrained_models/w2vec_noticias.model')

In [34]:
w2vec_2 = KeyedVectors.load('./pretrained_models/w2vec_noticias.model')

## Tasks: Palabras mas similares y Analogías

### Palabras mas similares

Tal como dijimos anteriormente, los embeddings son capaces de codificar toda la información contextual de las palabras en vectores.

Y como cualquier objeto matemático, estos pueden operados para encontrar ciertas propiedades. Tal es el caso de las  encontrar las palabras mas similares, lo que no es mas que encontrar los n vecinos mas cercanos del vector.  

In [36]:
w2vec_2.wv

In [35]:
w2vec_2.wv.most_similar(positive=["social"])

[('alcaz', 0.36716562509536743),
 ('pon', 0.3508830666542053),
 ('instanci', 0.3440763056278229),
 ('hoffm', 0.33845919370651245),
 ('urban', 0.3383106589317322),
 ('vaticin', 0.33535242080688477),
 ('gratuit', 0.332356333732605),
 ('middl', 0.33211106061935425),
 ('imped', 0.3308682441711426),
 ('ido', 0.3300718069076538)]